<a href="https://colab.research.google.com/github/seulmi0827/fininsight/blob/main/JACE/Jace_full_refine_sentence(%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90_%EC%B5%9C%EC%A2%85%EA%B2%B0%EA%B3%BC%EC%9A%A9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -q kss flashtext
!pip install -q pandarallel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 31.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.9/757.9 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 11.6 MB/s eta 0:00

In [2]:
!apt-get update
!apt-get install g++ openjdk-8-jdk -y
!pip install konlpy
!pip install mecab-python
!apt-get install curl -y
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

# import

In [3]:
from typing import List
import pandas as pd
import numpy as np
import re
from kss import split_sentences
from pandarallel import pandarallel
from konlpy.tag import Mecab

## 전처리 함수

In [48]:
def extract_aspect_sentences(df, std_threshold=2.0):
    pandarallel.initialize(progress_bar=True, verbose=1)

    def process_content(content):
        results = []
        try:
            # 문장 분리
            sentences = split_sentences(content)

            sentence_lengths = [len(sentence) for sentence in sentences]  # 문장의 길이(문자 수)
            mean_length = np.mean(sentence_lengths)  # 문장 길이의 평균값
            std_length = np.std(sentence_lengths)  # 문장 길이의 표준편차
            min_length = max(30, mean_length - std_threshold * std_length)  # 최소 길이 = 평균 - (표준편차 × 임계값), 최소 30자 보장
            max_length = mean_length + std_threshold * std_length  # 최대 길이 = 평균 + (표준편차 × 임계값)

            mecab = Mecab()
            for sentence in sentences:
                # 문장 길이 필터링
                if len(sentence) < min_length or len(sentence) > max_length:
                    continue

                # 기존 형태소 분석
                pos_tagged = mecab.pos(sentence)

                # 마침표 제거 를 통한 형태소 분석 준비
                sentence_without_dot = sentence.rstrip('.')
                pos_tagged_without_dot = mecab.pos(sentence_without_dot)

                if not pos_tagged:
                    continue

                # 자연스럽지 않은 문장 끝 필터링
                if (pos_tagged[-1][1].startswith('N') or      # 원본 문장 마지막이 명사인 경우
                    pos_tagged[-1][1].startswith('J') or      # 원본 문장 마지막이 조사인 경우
                    pos_tagged[-1][1].startswith('MM') or     # 원본 문장 마지막이 관형사인 경우
                    pos_tagged[-1][1].startswith('MA') or     # 원본 문장 마지막이 부사인 경우
                    pos_tagged[-1][1] == 'SN' or             # 원본 문장 마지막이 숫자인 경우
                    (pos_tagged_without_dot and (pos_tagged_without_dot[-1][1].startswith('N') or  # 마침표 제거 후 명사인 경우
                                                pos_tagged_without_dot[-1][1].startswith('J') or  # 마침표 제거 후 조사인 경우
                                                pos_tagged_without_dot[-1][1].startswith('MM') or # 마침표 제거 후 관형사인 경우
                                                pos_tagged_without_dot[-1][1].startswith('MA') or # 마침표 제거 후 부사인 경우
                                                pos_tagged_without_dot[-1][1] == 'SN'))          # 마침표 제거 후 숫자인 경우
                    ):
                    continue

                # 문장 복사
                processed_sentence = sentence

                i = 0
                while i < len(pos_tagged) - 1:
                    word, pos = pos_tagged[i]

                    # 현재 단어가 명사인 경우
                    if pos.startswith('N'):
                        next_idx = i + 1
                        josa_end_idx = i

                        # 명사 다음에 오는 연속된 조사 찾기
                        while next_idx < len(pos_tagged) and pos_tagged[next_idx][1].startswith('J'):
                            josa_end_idx = next_idx
                            next_idx += 1

                        # 조사가 있으면 모두 제거
                        if josa_end_idx > i:
                            noun_pos = processed_sentence.find(word)
                            if noun_pos != -1:
                                noun_end = noun_pos + len(word)
                                josa_start = noun_end
                                josa_end = noun_end

                                # 모든 조사의 길이를 더해 제거할 범위 계산
                                for j in range(i+1, josa_end_idx+1):
                                    josa_end += len(pos_tagged[j][0])

                                processed_sentence = processed_sentence[:josa_start] + processed_sentence[josa_end:]

                            # 다음 검사는 조사 다음부터
                            i = josa_end_idx + 1
                        else:
                            i += 1
                    else:
                        i += 1

                results.append({'raw_sentence': sentence, 'processed_sentence': processed_sentence})

        except Exception as e:
            print(f"오류 발생: {e}")

        return results

    # 병렬 처리
    all_results = df['content'].parallel_apply(process_content)

    # 결과 합치기
    flattened_results = [item for sublist in all_results for item in sublist]
    return pd.DataFrame(flattened_results)


text = [
# "안녕하세요"
# "가. 나. 다. 라. 마.",
# "안녕하세요, 테스트합니다. 물음표 테스트 할게요? 할게요."
# "띄   어쓰기    테스트 입    니다.      ",
# "22일(현지시간) 미국 새너제이 SAP센터에서 열린 상반기 '갤럭시 언팩 2025'에서 전시된 XR 기기 '프로젝트 무한",
# "삼성전자가 AI폰 '갤럭시 S25 시리즈'를 만날 수 있는 '갤럭시 스튜디오'를 오픈한다고 24일 밝혔다. 사진은 '갤럭시 스튜디오' 내부 전경.",
# "삼성전자, 작년 4분기 매출 75조7883억원 영업익 6조4927억원",
# "삼성전자, 지난해 영업익 32조7000억원. 삼성전자가 지난해 4분기 반도체 사업에서 2조9천억원의 영업이익을 내는 데 그쳤다.",
# "삼성전자 ""4분기 HBM 매출 1.9배 증가""",
# "딥시크 충격 피할 종목 찾아라.' 중국 스타트업 '딥시크(DeepSeek)'발 충격으로 서학개미들이 혼란에 휩싸였다.",
# "수요둔화에 전분기比 24% 뚝 지난해 전체 영업이익 32.7조 딥시크 충격 더해 주가 2%↓ SK하이닉스는 9.8% 급락 삼성전자 반도체가 다시 뒷걸음질했다.",
# "이 기존 내외부망을 분리하는 보안 체계만으로는 AI 기반의 새로운 위협을 완전히 막을 수 없이 판단에 따른 것으로 보인다."
# "이 학생과 교사에게 PC, 태블릿 등을 제공해 교육이 중단 없이 지속되도록 돕는다."
"주간 AI PRISM  맞춤형 주간 경제 브리핑   편집자 주   'AI PRISM'(Personalized Report   Insight Summarizing Media)은 한국언론진흥재단의 지원을 받아 개발한 '인공지능(AI) 기반 맞춤형 뉴스 추천 및 요약 서비스'입니다. "
"삼성전자(005930)가 미국 로스앤젤레스(LA)에서 발생한 대형 산불로 피해를 본 지역사회 및 주민들을 위해 100만 달러(약 14억5000만 원) 규모의 가전 태블릿 등 전자제품을 기부한다. 7일 삼성전자 미국 뉴스룸에 따르면 삼성전자 미국 법인과 삼성반도체는 해비타트 오브 그레이터 로스앤젤레스, 팀 루비콘, 로스앤젤레스 통합 교육구 교육재단 등과 전략적 파트너십을 맺고 LA 대형 산불 피해를 본 지역사회 재건을 지원한다. 삼성전자는 우선 보험이 없거나 소득이 낮은 시민들을 대상으로 필수적인 가전제품을 지원한다. 또 재난 현장에서 봉사하는 재향군인 중심 조직 팀 루비콘에 첨단 통신 장치를 제공해 현장에서 정부 기관이나 자원봉사자들과 원활하게 협력할 수 있도록 돕는다. 한편 지난달 7일부터 LA 지역에서 발생한 2건의 산불로 최소 28명이 숨졌고 1만 6000개 이상의 건물이 소실돼 수십만 명의 이재민이 발생했다."
"우리는 혼돈의 시기 한복판에 살고 있다. 인공지능 열풍에 글로벌 판도가 바뀌고 있으며, 트럼프 행정부 출범에 전 세계 시장이 출렁이고 있고, 국내 정치는 한 치 앞도 내다보기 어렵다. 검찰은 이런 시기에 고민이 클 것이다. 2심에서 19개 혐의에 대해 모두 무죄를 선고받은 삼성전자 이재용 회장을 대법원에 상고할지를 며칠 내에 결정해야 해서다. 이제 중요한 질문을 해볼 때다. 상고한다면 정의를 위한 것일까, 아니면 정치적 패턴의 반복일까. 그동안 한국 정치는 법리에 군림하는 일이 잦았다. 상대 진영의 척결을 알리는 신호탄으로, 권력 교체기의 길들이기 일환으로 기업인은 이용당하기 일쑤였다. 정권이 바뀔 때 마다 특정 인물이 법정에 서는 익숙한 풍경. 정의를 위한 노력도 일부 있었겠지만, 정치 메커니즘이 작동한 사례도 많았다. 자본주의가 고도로 발달한 미국은 전혀 다른 접근 방식을 취한다. 법은 기업인에 대한 책임을 물으면서도 개인에 대한 형사 기소보다는 규제 조치와 금전적 처벌을 주로 사용한다. 이는 법의 궁극적 목표는 형벌 그 자체가 아니라, 사회적 문제를 효과적으로 해결하는 데 있다고 믿은 윌리엄 제임스와 존 듀이의 실용주의적 법철학이 깔려 있기 때문이다. 때문에 최고경영자(CEO) 개인이 소송의 피고로 지목되는 사례는 드물다. 순다르 피차이, 일론 머스크, 팀 쿡 등이 미국 연방검찰에 기소 받아 실형을 선고받았다는 뉴스를 들어본 적이 있는가. 마크 저커버그 메타 CEO가 미성년자의 소셜 미디어 중독을 방치했다는 혐의로 소송을 당했지만, 법원이 CEO 개인에게 책임을 물을 수 없다고 판단해 소송을 기각한 사례는 있다. 때문에 검찰의 상고는 이재용 개인이 유죄냐 무죄냐 차원이 아니다. 더 중요한 질문은 이것이다. 한국의 법적 시스템은 일관되고 예측할 수 있는 방식으로 운영되고 있는가. 사법 정의를 준수하면서도 국가 경제의 안정성을 유지할 수 있는가. 상고가 정의를 위한 것이라면, 그 논리가 명확해야 한다. 하지만 그것이 기계적 관행의 반복이라면 곤란하다. 다시 한번 생각해볼 필요가 있다."
"삼성전자가 미국 로스앤젤레스에서 발생한 대형 산불로 피해를 본 지역사회 복구 지원을 위해 100만달러, 우리돈 약 14억5천만원 규모의 가전 태블릿PC 제품을 기부합니다. 삼성전자 미국법인과 삼성 반도체는 해비타트 오브 그레이터 로스앤젤레스, 팀 루비콘, 로스앤젤레스 통합 교육구 교육재단 등과 전략적 파트너십을 통해 지원에 나서기로 했습니다. 저소득층을 중심으로는 가전제품을, 교육 중단 상황이 발생하지 않도록 학생과 교사에겐 PC와 태블릿 기기를 지원합니다. 또 현장 복구작업을 진행 중인 자원봉사자들에게도 첨단 통신 장치를 제공할 계획입니다. 지난달 7일 LA 지역의 서부 해변과 동부 내륙에서 발생한 2건의 산불로 최소 28명이 사망하고 22명이 실종 신고됐으며, 수십만 명의 이재민이 발생했습니다. 삼성전자는 ""복구가 길고 어려운 과정이 될 것이라는 점을 알고 있다""며 ""지역 사회가 재건하고 번영하는 데 필요한 자원을 확보할 수 있도록 지속적인 지원 제공을 위해 최선을 다하고 있다""고 전했습니다.  삼성전자  LA  산불  기사문의 및 제보   카톡 라인 jebo23"
"삼성전자가 미국 로스엔젤레스에서 발생한 산불로 피해를 입은 지역 사회 구호 및 복구 노력을 지원하기 위해 100만 달러 규모의 제품을 기부하기로 했다고 6일 밝혔다. 삼성전자는 해비타트, 팀 루비콘, 로스엔젤레스 통합교육구 교육재단 등과 전략적 파트너십을 통해 피해를 입은 가정과 지역사회에 즉각적인 구호와 장기적인 재건 지원을 제공하는 것을 목표로 하고 있다. 지난 1월 7일 미국 캘리포니아주 로스앤젤레스주 일대에서 대규모 산불이 발생해 약 3주간 퍼시픽 팰리세이드, 알타데나 등 여러 지역이 피해를 입었다. 최근 지역사회는 20일 넘게 넘게 이어진 산불로 전소된 약 1만 2000채의 주택 등에서 잔해를 치우는 작업을 실시했다. 이에 삼성전자는 해비타트를 통해 필수 가전제품을 기부해 지역사회 재건을 도울 방침이다. 아울러 LASUD 교육재단과 협력해 학생과 교사에게 PC, 태블릿 및 기타 도구를 제공해 위기상황에서도 교육이 중단 없이 계속될 수 있도록 돕는다. 삼성전자는 재난 전, 재난 중, 재난 후에 봉사하는 베테랑 주도 조직인 팀 루비콘에 첨단 통신 장치를 제공한다. 해당 장치들은 미국 정부 기관, LA 소방서, 지역 비상 관리자 및 캘리포니아 전역의 20만명 이상의 지역 자원 봉사자에게 전달된다. 해비타트 관계자는 ""이러한 규모의 재해로부터 복구하는 것은 긴 여정이 될 것""이라며 ""미래에 대한 희망을 가지고 재건할 수 있도록 필수 자원을 제공하는 삼성의 지원에 감사드린다""고 말했다."
"이재명 추진에 기대감 높아졌던 '주52시간 근로제한 예외' 적용 당내 산자 환노위원 반대 좌초위기 '뒤쳐진 법'에 국가 경쟁력 발목 반도체 공정현장 ［헤럴드 DB  '52시간 근로 제한 예외조항'을 적용하는 문제를 놓고 더불어민주당 내부 의견이 엇갈리면서 반도체특별법 국회 통과가 난항을 겪고 있다. 당초 반도체 패권을 회복하자는 결의 속 해당 산업에 한해 52시간 근무에 유연성을 부과하자는 취지로 추진됐지만 민주당 내 우경화 비판으로 이 조항이 제외된 안이 처리될 가능성이 높아지고 있다. 하지만 반도체법의 핵심은 52시간 근무 예외를 명문화, 근무의 관성과 가속이 필요한 기술 인력의 연구 시간에 유연성을 부여하자는 것이었기 때문에 이 조항이 빠질 경우 '앙꼬 없는 찐빵' 이 될 수밖에 없다는 지적이 나온다. 일각에서는 이럴 경우 법이 시대변화를 반영하지 못하는 또 하나의 사례가 될 수 있다는 비판도 제기된다.  관련기사 6면 민주당은 현재 반도체법에 '주 52시간 근로제한 예외 조항'을 분리해 지원 방안 위주로 처리하자는 입장이다. '화이트칼라 이그젬션'이라 불리는 고소득 연구직 등의 주 52시간 근로제한 예외 조항은 여야 뿐 아니라 각 당내에서도 세부사항을 두고 여러 이견이 있는 상황이다. 하지만 경제계는 반도체 등 핵심 기간산업에서 '화이트칼라 이그젬션' 도입이 필요하다는 입장이다. 삼성전자 임원 출신의 반도체 업계 관계자는 ""그야말로 법이 시대변화를 따라가지 못하고 있는 전형적 사례""라며 ""반도체를 포함한 여러 국가핵심산업의 연구개발은 하루 8시간, 주 40시간 이런 식으로 칼로 무 자르듯 정량화할 수 있는 것이 아니다""라고 말했다. 이어 ""과거 1980~90년대엔 근로자가 자신의 노동에 대한 합당한 보상을 받지 못하는 등 처우가 열악했지만 지금은 노동환경도 인구구조도 변했다""며 ""주52시간 근로제한은 좀 더 열심히 일해서 성과를 내고 싶은 인재들의 경쟁력을 저해하는 '걸림돌'이 되고 있다""고 덧붙였다. 현재 주 52시간 근무제는 일주일 동안 기본 근로시간 40시간을 정해두고 연장근로를 12시간까지 허용한다. 산업계에서는 주 단위 근로시간 산정 기준을 월이나 분기 등으로 변경해야 한다는 주장이다. 이재명 민주당 대표는 3일 토론회에서 ""특정 산업의 연구 개발 분야 고소득 전문가가 동의할 경우 예외로 몰아서 일하게 해주자는 게 왜 안 되냐 하니 할 말이 없더라""라며 ""저는 기본적으로 노동시간제에 예외를 두지 않는 게 좋다고 생각하지만, 이 점을 설득하지 못하면 '그게 왜 안 되지'라는 생각이 든다""고 말한 바 있다. 하지만 이 대표는 당내 '우클릭' 비판에 막혀 자신의 생각을 관철시키지 못하는 상태다. 전문가들은 ""반도체는 시간싸움""이라며 주52시간 근로제한이 집중해서 연구해야 하는 실제 현장과 맞지 않다고 입을 모은다. 지난해 12월 한국공학한림원 반도체특별위원회가 10개월 간의 연구결과를 발표하는 자리에서 안현 SK하이닉스 개발총괄 사장도 ""개발은 하다 보면 관성이 붙고 가속이 붙어서 쭉 가야 한다""며 ""52시간 자체는 좋은 제도지만, 개발이란 특수적 활동에 있어서는 그것이 부정적인 어떤 습관이나 관행을 만들지 않을까 하는 우려가 있다""고 말했다. 이혁재 서울대 교수도 ""30분만 더 일하면 결과를 얻을 수 있는데 퇴근을 해야 하니까 다음날 다시 일을 처음부터 하게 되고, 그러니 기술 개발 속도가 점점 느려지고 있다""고 지적했다. TSMC, 엔비디아 등 다른 글로벌 업체들은 유연한 근로 시스템을 기반으로 연구 개발(R D)에 속도를 내고 있다. 블룸버그통신은 지난해 8월 엔비디아 전현직 직원 10명을 인터뷰해 고강도 업무 환경을 조명했다. 주 7일 연속 근무나 새벽 1~2시 근무 사례가 특이하지 않으며, 이들이 주식 등 파격적인 금전 보상을 받고 있어 이직률이 업계 평균보다 낮다는 보도였다. 미국은 주 40시간 법정 근로시간을 운영 중이지만, 연장근로시간에 제한을 두지 않고 추가근로시간에 대해 정규 임금의 최소 1.5배를 지급한다. 또  고위관리직 전문직 컴퓨터직 등에 종사하면서 주 684달러 이상을 버는 근로자  연소득 10만7432달러 이상인 근로자는 근로시간 규제에서 제외하는 화이트 칼라 이그젬션을 시행 중이다. 주 40시간제를 채택한 대만은 노사가 합의하면 하루 근무를 8~12시간 늘릴 수 있도록 했다. TSMC의 R D 조직이 24시간, 주 7일 돌아갈 수 있는 배경이다."
"첫 공장 27년 5월 완공 목표 대규모 사업 본격화 토지 이어 오피스 시장 지형 변화 예상 용인 반도체 메가 클러스터에서 내달 첫 생산시설(팹 Fab) 착공을 비롯한 대규모 사업이 본격화되면서 인근 부동산 시장에 미치는 영향도 주목받고 있다. 7일 업계에 따르면 용인시는 SK하이닉스의 첫 반도체 생산라인 공사에 필요한 장비 선정을 위한 업체 설명회를 오는 13일 열 계획이다. 시공사인 SK에코플랜트와 협약에 따라 크레인 고소작업차 지게차 등의 지역 업체를 우선 선정하는 절차다. SK하이닉스는 용인 처인구 원삼면 일반산업단지(415만여㎡126만평)에 첫 반도체 공장을 2027년 5월 완공하고 나머지 세 개 팹도 순차적으로 건설해 반도체 거점으로 성장시킨다는 계획이다. 또 1조원 투자규모의 미니팹(테스트베드) 구축사업도 본격화될 예정이다. 반도체 대량생산 시설과 동일한 환경에서 소부장(소재 부품 장비) 기업들이 개발한 제품의 신뢰성을 검증하는 시설이다. 용인 처인구 이동 남사읍 일대 778만여㎡(235만평)에 지난해 말 지정 고시된 삼성전자 중심의 첨단시스템반도체 국가산업단지에서는 올해 보상과 이주절차에 들어간다. 보상금이 풀리면 주변 부동산 가격을 자극할 것이란 예상도 나오고 있다. 정부와 용인시는 반도체 클러스터로 연결되는 도로 철도 용수 등 인프라 확충에 속도전을 낼 전망이다. 환경부는 용수공급 1단계(하루 31만 톤) 구간에 대한 기본 및 실시설계를 우선 착수한다고 밝혔다. 국토교통부는 반도체 국가산단 조성계획에서 발표한  국도 45호선 8차로 확장  반도체 고속도로 신설(45.3km)  국지도 82호선 신설 확장 등의 후속 조치에 나서게 된다. 용인시는 올해 지방도 신설 확장 및 경기남부광역철도 신설 추진 사업 등에 4521억원을 투입할 계획이다. 용인 반도체 클러스터 및 주변 인프라 조성에 탄력이 붙을 것으로 예상되면서 부동산 업계는 낙수효과도 기대하고 있다. 특히 반도체 공장 착공과 관련해 소부장 관련 기업들의 사무실 수요도 늘어날 것이란 예상도 나오고 있다. 현대엔지니어링이 용인시 기흥구 영덕동 삼성삼거리 인근에 분양 중인 지식산업센터 '신광교 클라우드 시티'도 수혜 단지로 꼽힌다. 단지 앞을 지나는 중부대로(국도 42번)가 용인 반도체 클러스터로 이어지는 입지여건에다 대규모로 조성돼 입주기업간 시너지 효과도 기대되기 때문이다. 단지는 지하 6층, 지상 최대 33층, 5개 동으로 이뤄지며 연면적(35만여㎡)이 서울 롯데월드타워보다 큰 규모로 조성된다. 호텔급 커뮤니티시설에 컨시어지 서비스까지 제공하는 새로운 개념의 비즈니스 공간으로 조성되는 게 특징이다. 업계 관계자는 ""반도체 클러스터 조성사업과 관련해 용인시는 올해 1321개의 일자리 창출을 발표했다""며 ""반도체 공장 착공에 따라 업무 물류시설 수요가 크게 늘어날 것으로 예상된다""고 말했다. 발로 뛰는 는 24시간 여러분의 제보를 기다립니다.  카카오톡  '제보' 검색  이메일    뉴스 홈페이지   bbs report write"
"삼성전자가 7일부터 신작 '갤럭시 S25 시리즈'를 전 세계 주요 국가에서 출시한다. 삼성전자는 이날 한국과 미국, 영국, 인도, 태국 등을 시작으로 전 세계 120여개 국가에 S25를 순차 출시한다고 밝혔다. 출시에 맞춰 구글의 AI 제미나이를 사용할 수 있는 언어도 총 46개로 확대된다. MX(모바일경험)노태문  사업부장(사장)은 ""갤럭시 S25 시리즈는 사용자의 일상 자체를 혁신할 것""이라며 ""한층 더 발전한 갤럭시 인공지능(AI)을 통해 역대 가장 자연스럽고 개인화된 모바일 경험을 제공할 것""이라고 밝혔다. 삼성전자는 지난달 미국 캘리포니아주 새너제이에서 '갤럭시 언팩 2025' 행사를 열고 갤럭시 S25 시리즈를 공개했다. 역대 갤럭시 최초로 통합형 AI 플랫폼 'One UI 7'을 탑재한 이번 시리즈는 텍스트, 음성, 이미지 등 다양한 데이터를 이해하고 사용자 취향을 분석해 여러 앱을 넘나드는 자연스러운 AI 경험을 구현한다. 스마트폰 측면 버튼을 눌러 AI 에이전트를 호출하면 친구와 대화하듯 자연스럽게 말로 명령어를 입력하고 작업을 처리할 수 있다. 삼성전자는 2월 한 달 동안 갤럭시 S25 시리즈 국내 구매 고객을 대상으로 삼성닷컴 앱에서 '갤럭시 워치7' 구매 시 사용할 수 있는 15만원 할인 쿠폰, 유튜브 프리미엄 3개월 무료 체험 등 제휴 콘텐츠 혜택을 제공한다. '구글 원 AI 프리미엄' 6개월 무료 구독권도 제공해, 제미나이 어드밴스드와 2TB 구글 드라이브 스토리지 체험도 가능하다. '뉴 갤럭시 AI 구독클럽' 가입도 지난달 24일부터 시작했다. 구독 클럽을 통해 S25 시리즈 자급제 모델을 구입하면 기기 반납 시 최대 50% 잔존가 보상, '삼성케어플러스 스마트폰 파손+' 제공, 모바일 액세서리 할인 등 혜택을 제공한다. 가입 기간은 12개월, 24개월 중 선택할 수 있고, 구독료는 월 5900원이다.  는 여러분의 제보로 함께 세상을 바꿉니다. 각종 비리와 부당대우, 사건사고와 미담 등 모든 얘깃거리를 알려주세요. 이메일    카카오톡     사이트    b71afn"
"국내 방산기업들도 중국의 생성형 인공지능(AI) '딥시크' 차단에 동참했다. 국가 안보와 밀접하게 관련된 만큼 강력한 보안 조치를 시행키로 한 것이다. 기존 내외부망을 분리하는 보안 체계만으로는 AI 기반의 새로운 위협을 완전히 막을 수 없다는 판단에 따른 것으로 보인다. 7일 방산업계에 따르면 SNT그룹의 계열사인 SNT다이나믹스, SNT모티브는 최근 내부 공지를 통해 ""딥시크가 광범위하게 수집하는 이용자의 데이터를 중국으로 유출할 우려가 있다""며 사내 접속을 차단했다. 개인용 통신기기 등에서도 사용을 자제해 달라면서 ""개인용 정보통신기기를 접속할 땐 사내 사업정보나 기술자료 등이 입력되지 않도록 주의해 달라""고 공지했다. 현대로템과 STX엔진도 같은 날 내부 공지를 통해 차단을 알렸다. 국가 안보와 기밀 보호를 위한 조치로, 보안 강화를 최우선으로 고려한 결정이다. 한화에어로스페이스, 한화시스템 등은 이미 지난해부터 사내 컴퓨터에서 생성형 AI를 설치하지 못하도록 조치한 바 있다. 국내 대기업들은 딥시크 접속 차단과 관련해 별도의 조치를 내놓지 않고 있다. 이미 2023년 챗GPT 돌풍이 일어날 때부터 외부의 생성형 AI 접속을 제한한 만큼 기존 입장에서 크게 달라질 게 없다는 이유다. 보안 문제와 데이터 유출 우려에서 자유롭기 위해 AI 시스템 자체 개발에 주력하는 성향이 강해지고 있다. 업계 관계자는 ""딥시크를 포함한 외부 AI 서비스를 이용하면 입력된 정보가 어디에 저장되는지 명확하지 않다""며 ""영업기밀이나 중요 기술 정보가 유출될 가능성이 있어 내부 AI 시스템을 강화하는 것이 최선""이라고 말했다. 이에 따라 삼성 등 대기업들은 자체 개발한 'AI 비서'로 직원들의 연구개발(R D)과 업무를 지원하고 있다. AI 내재화 전략이 산업계 경쟁력 강화로 이어질 것이란 기대감도 나온다. 삼성전자는 2023년 말 공개한 업무용 AI '가우스'가 있다. 최근 서로 다른 유형의 데이터를 동시 처리할 수 있는 '멀티모달' 가우스2로 업그레이드됐다. LG는 업무용 AI '엘지니'와 R D 중심의 '엑사원'이 있다. LS그룹은 오픈AI GPT 모델을 기반으로 개발한 'LS GPT'를 운용한다. 정보기술(IT) 업계도 자체 AI 프로그램을 쓰고 있다. 샘 올트먼 오픈AI 최고경영자(CEO)의 방한으로 '동맹 전선'을 구축한 카카오도 딥시크 금지령을 내렸다. 사내에선 'AI 버디'가 역할을 대신한다. 네이버는 '클로바(CLOVA)'를 통해 사내 업무의 효율성과 생산성을 높이고 있다. 업계 관계자는 ""AI 기술이 발전할수록 보안 위협도 함께 증가하고 있다""며 ""국내 기업들은 앞으로 자체 AI 개발을 더욱 강화해 외부 의존도를 줄이는 방향으로 나아가야 한다""고 했다. 한편 딥시크는 다른 생성형 AI 서비스와 비교할 때 이용자 데이터를 과도하게 수집한다는 지적을 받는다. 개인정보 약관을 보면 프롬프트에 이용자가 직접 입력하는 정보 외에 기기 정보, 일련번호, IP 주소 등을 수집한다. 심지어는 키보드 입력 패턴과 리듬 정보까지 축적한다."
"갤럭시S25 시리즈가 전 세계에서 순차 출시된다. 한국을 포함해 미국, 영국, 인도, 태국을 시작으로 전 세계 120여개국에서 선보일 예정이다. 심성전자는 7일 갤럭시S25 시리즈를 전 세계에서 순차적으로 출시한다고 밝혔다. 이날 한국과 미국, 영국, 인도, 태국 등에서 먼저 출시한 뒤 이어서 순차 출시한다는 계획이다. 갤럭시S25 시리즈는 이달 3일까지 진행한 국내 사전 판매에서 130만대를 기록했다. 이는 역대 갤럭시S 시리즈 사전 판매 중 최다 기록이다. 121만대를 기록했던 갤럭시S24 시리즈가 그간 갤럭시S 시리즈 최다 사전 판매 기록은 121만대를 기록한 갤럭시S24 시리즈였다. 사전 예약자 2명 중 1명은 울트라 기종을 선택했다. 전작인 S24 시리즈 사전 예약 기간에도 울트라 예약자는 65%를 차지했다. 기종별로 사전 예약자 비중을 보면 울트라 52%기본형 26%플러스 22% 순이었다. 사전 예약자들은 갤럭시S25와 갤럭시S25 플러스의 경우 아이스블루 색상을 선호했다. 갤럭시S25 울트라는 티타늄 실버블루와 티타늄 블랙의 인기가 높았다. 갤럭시S25 시리즈는 역대 갤럭시 최초로 통합형 인공지능(AI) 플랫폼인 '원(One) UI 7'을 사용한다. 나우 브리프 기능을 통해 일상 속 맞춤 정보를 받아볼 수 있다. 나우 바로 잠금 화면에서도 나우 브리프 내용을 확인할 수 있다. 갤럭시 전용 프로세서 '스냅드래곤 8 엘리트'로 성능도 끌어올렸다. 해당 칩셋은 디스플레이 품질과 효율성을 극대화하는 방향으로 설계됐다. 카메라 성능 또한 향상됐다. 고해상도 센서와 AI 기반의 차세대 '프로비주얼 엔진'이 있어 어느 거리에서도 고화질 사진을 촬영할 수 있다. AI로 만든 콘텐츠에 출처를 남길 수도 있다. 갤럭시S25 시리즈는 스마트폰 최초로 AI로 생성된 이미지에 '콘텐츠 자격증명'기술을 적용했다. AI로 생성된 이미지 데이터에 편집 이력을 남겨 콘텐츠 출처 및 편집 여부를 기록한다. 구글 제미나이를 사용할 수 있는 언어도 총 46개로 확대됐다. 삼성전자는 2월 한 달간 갤럭시S25 시리즈 국내 구매 고객을 대상으로 삼성닷컴 앱에서 '갤럭시 워치7' 구매할 때 사용할 수 있는 15만원 할인쿠폰, 유튜브 프리미엄 3개월 무료 체험 등 제휴 콘텐츠 혜택을 제공한다. 구글 원 AI 프리미엄 6개월 무료 구독권도 제공한다. 제미나이 어드벤스와 2TB 구글 드라이브 스토리지 체험도 가능하다. 삼성전자는 새로운 구독 프로그램 '뉴 갤럭시 AI 구독클럽' 가입도 지난달 24일부터 시작했다. 해당 구독 상품으로 갤럭시S25 시리즈 자급제 모델을 구입하면  기기 반납 시 최대 50% 잔존가 보상  '삼성케어플러스 스마트폰 파손+' 제공  모바일 액세서리 할인 등 혜택을 제공한다. 가입 기간은 12개월, 24개월 중 선택 가능하며 구독료는 월 5900원이다. 삼성전자는 갤럭시S25 시리즈 사전 예약 고객 대상 개통 마감일을 이달 12일까지로 늘렸다. 삼성닷컴 공지사항에 따르면 지난 4일부터 시작한 사전 개통 기간의 마감일을 기존 7일에서 12일까지로 5일 연장했다. 일부 온오프라인 판매처에서 준비된 물량의 사전 판매가 완료되면서 고객 불편을 최소화하기 위한 조치다. 노태문 삼성전자 모바일경험(MX)사업부장(사장)은 ""갤럭시 S25 시리즈는 사용자의 일상 자체를 혁신할 것""이라며 ""한층 더 발전한 갤럭시 AI를 통해 역대 가장 자연스럽고 개인화된 모바일 경험을 제공할 것""이라고 말했다."
"삼성전자(대표 한종희)가 7일부터 '갤럭시 S25 시리즈'를 전 세계 주요 국가에서 출시한다. 삼성전자는 이날 한국, 미국, 영국, 인도, 태국 등을 시작으로 전 세계 120여개국에 갤럭시 S25 시리즈를 순차 출시한다고 발표했다. 노태문 삼성전자 MX사업부장 사장은 ""갤럭시 S25 시리즈는 사용자의 일상 자체를 혁신할 것""이라며, ""한층 더 발전한 갤럭시 AI를 통해 역대 가장 자연스럽고 개인화된 모바일 경험을 제공할 것""이라고 말했다. 이번 갤럭시 S25 시리즈 출시에 맞춰 구글 제미나이를 사용할 수 있는 언어가 총 46개로 확대된다. 역대 갤럭시 최초로 통합형 AI 플랫폼인 '원 UI 7' 기반 텍스트, 음성, 이미지 등 다양한 사용자 접점에서 상황을 이해하고 취향을 분석해 여러 앱 간 자연스러운 AI 경험을 구현한다. 이번에 새롭게 탑재된 '나우 브리프(Now Brief)' 기능은 일상 속 맞춤형 정보 브리핑을 제공하고, '나우 바(Now Bar)'를 통해 사용자는 잠금 화면에서도 손쉽게 활동을 확인할 수 있다. 또한 스마트폰 측면 버튼을 눌러 AI 에이전트를 호출해, 친구와 대화하듯 자연스럽게 말로 명령어를 입력하고 작업을 처리할 수 있다. 구글과의 협업으로 갤럭시 S24 시리즈에 처음 탑재됐던 '서클 투 서치' 기능도 진화했다. 다양한 데이터 형태를 처리할 수 있는 멀티모달이 적용돼 기존의 이미지, 텍스트 검색에 더해 기기에서 재생되는 사운드 검색도 지원한다. 갤럭시 S25 시리즈는 스마트폰 최초로 AI로 생성된 이미지에 '콘텐츠 자격증명'기술을 적용했다. AI로 생성된 이미지 데이터에 편집 이력을 남겨 콘텐츠 출처 및 편집 여부를 기록한다. 한편, 삼성전자는 2월 한 달간 갤럭시 S25 시리즈 국내 구매 고객 대상 삼성닷컴 앱에서 갤럭시 워치7 구매 시 사용할 수 있는 15만원 할인 쿠폰, 유튜브 프리미엄 3개월 무료 체험 등 제휴 콘텐츠 혜택을 제공한다. 구글 원 AI 프리미엄 6개월 무료 구독권도 제공해 제미나이 제미나이 어드밴스드와 2TB 구글 드라이브 스토리지 체험도 가능하다. 삼성전자는 '뉴 갤럭시 AI 구독클럽' 가입도 지난달 24일부터 시작했다. 갤럭시 S25 시리즈 자급제 모델을 구입한 뒤, 해당 구독 상품에 가입할 수 있다. 혜택으로는  기기 반납 시 최대 50% 잔존가 보상  '삼성케어플러스 스마트폰 파손+' 제공  모바일 액세서리 할인 등이 제공된다. 가입 기간은 12개월, 24개월 중 선택 가능하며 구독료는 월 5900원이다."
"반도체법 '주52시간 근로제한 예외' 없이 통과 가능성 산업계, ""반도체뿐 아니라 핵심산업 전반 '걸림돌'"" ""법이 시대변화 못 따라가 무한경쟁 시대서 뒤처질 것"" 반도체 공정현장［DB］  '52시간 근로 제한 예외조항'을 적용하는 문제를 놓고 더불어민주당 내부 의견이 엇갈리면서 반도체특별법 국회 통과가 난항을 겪고 있다. 당초 반도체 패권을 회복하자는 결의 속 해당 산업에 한해 52시간 근무에 유연성을 부과하자는 취지로 추진됐지만 민주당 내 우경화 비판으로 이 조항이 제외된 안이 처리될 가능성이 높아지고 있다. 하지만 반도체법의 핵심은 52시간 근무 예외를 명문화, 근무의 관성과 가속이 필요한 기술 인력들의 연구 시간에 유연성을 부여하자는 것이었기 때문에 이 조항이 빠질 경우 '앙꼬 없는 찐빵' 이 될 수 밖에 없다는 지적이 나온다. 일각에서는 이럴 경우 법이 시대변화를 반영하지 못하는 또 하나의 사례가 될 수 있다는 비판도 제기된다. 민주당은 현재 반도체법에 '주 52시간 근로제한 예외 조항'을 분리해 지원 방안 위주로 처리하자는 입장이다. '화이트칼라 이그젬션'이라 불리는 고소득 연구직 등의 주 52시간 근로제한 예외 조항은 여야 뿐 아니라 각 당내에서도 세부사항을 두고 여러 이견이 있는 상황이다. 하지만 경제계는 반도체 등 핵심 기간산업에서 '화이트칼라 이그젬션' 도입이 필요하다는 입장이다. 삼성전자 임원 출신의 반도체 업계 관계자는 ""그야말로 법이 시대변화를 따라가지 못하고 있는 전형적 사례""라며 ""반도체를 포함한 여러 국가핵심산업의 연구개발은 하루 8시간, 주 40시간 이런 식으로 칼로 무 자르듯 정량화할 수 있는 것이 아니다""라고 말했다. 이어 ""과거 1980~90년대엔 근로자들이 자신의 노동에 대한 합당한 보상을 받지 못하는 등 처우가 열악했지만 지금은 노동환경도 인구구조도 변했다""며 ""주52시간 근로제한은 좀 더 열심히 일해서 성과를 내고 싶은 인재들의 경쟁력을 저해하는 '걸림돌'이 되고 있다""고 말했다. 현재 주 52시간 근무제는 일주일 동안 기본 근로시간 40시간을 정해두고 연장근로를 12시간까지 허용한다. 산업계에서는 주 단위 근로시간 산정 기준을 월이나 분기 등으로 변경해야 한다는 주장이다. 이재명 민주당 대표는 지난 3일 토론회에서 ""특정 산업의 연구 개발 분야 고소득 전문가들이 동의할 경우 예외로 몰아서 일하게 해주자는 게 왜 안 되냐 하니 할 말이 없더라""라며 ""저는 기본적으로 노동시간제에 예외를 두지 않는 게 좋다고 생각하지만, 이 점을 설득하지 못하면 '그게 왜 안 되지'라는 생각이 든다""고 말한 바 있다. 하지만 이 대표는 당내 '우클릭' 비판에 막혀 자신의 생각을 관철시키지 못하는 상태다. 전문가들은 ""반도체는 시간싸움""이라며 주52시간 근로제한이 집중해서 연구해야 하는 실제 현장과 맞지 않다고 입을 모은다. 지난해 12월 한국공학한림원 반도체특별위원회가 10개월 간의 연구결과를 발표하는 자리에서 안현 SK하이닉스 개발총괄 사장도 ""개발은 하다 보면 관성이 붙고 가속이 붙어서 쭉 가야 한다""며 ""52시간 자체는 좋은 제도지만, 개발이란 특수적 활동에 있어서는 그것이 부정적인 어떤 습관이나 관행을 만들지 않을까 하는 우려가 있다""고 말했다. 이혁재 서울대 교수도 ""30분만 더 일하면 결과를 얻을 수 있는데 퇴근을 해야 하니까 다음날 다시 일을 처음부터 하게 되고, 그러니 기술 개발 속도가 점점 느려지고 있다""고 지적했다. TSMC, 엔비디아 등 다른 글로벌 업체들은 유연한 근로 시스템을 기반으로 R D에 속도를 내고 있다. 블룸버그통신은 지난 8월 엔비디아 전현직 직원 10명을 인터뷰해 고강도 업무 환경을 조명했다. 주 7일 연속 근무나 새벽 1~2시 근무 사례가 특이하지 않으며, 이들이 주식 등 파격적인 금전 보상을 받고 있어 이직률이 업계 평균보다 낮다는 보도였다. 미국은 주 40시간 법정 근로시간을 운영 중이지만, 연장근로시간에 제한을 두지 않고 추가근로시간에 대해 정규 임금의 최소 1.5배를 지급한다. 또한  고위관리직 전문직 컴퓨터직 등에 종사하면서 주 684달러 이상을 버는 근로자  연소득 10만7432달러 이상인 근로자는 근로시간 규제에서 제외하는 화이트 칼라 이그젬션을 시행 중이다. 주 40시간제를 채택한 대만은 노사가 합의하면 하루 근무를 8~12시간 늘릴 수 있도록 했다. TSMC의 R D 조직이 24시간, 주 7일 돌아갈 수 있는 배경이다."
"로봇 업체들이 국내 증시에서 꾸준한 상승세를 보이고 있다. 대기업발 투자 소식에 더해 미국 내 제조시설 투자 확대 등 사업 기대감이 투자자들의 관심에 불을 지폈다. 7일 오전 코스닥 시장에서 레인보우로보틱스는 전일 대비 1만1000원(2.94%) 오른 38만4천500원에 거래되고 있다. 레인보우로보틱스 주가는 올해 들어서 2배 이상 상승했다. 지난달 31일부터는 5거래일 연속 신고가를 기록했다. 시가총액은 7조4천억원까지 올라 코스닥 전체 5위다. 4위인 에코프로(8조원)와도 격차를 좁혔다. 지능형 로봇 서비스 전문기업 클로봇은 이번 주에만 2배 가까이 상승했다. 지난 5일 상한가에 이어 6일 12.96%7일 오전에도 20% 넘는 상승폭을 보이고 있다. 비전 인공지능(AI) 로봇 솔루션 기업 씨메스도 16% 상승하는 등 연일 강세다. 로보티즈(39,600원  800 +2.06%), 뉴로메카(34,300원  450 +1.33%), 엔젤로보틱스(31,300원  1,150 +3.81%) 등 로봇 제조사들과 에스피지(32,650원  400 +1.24%), 에스비비테크(27,500원  50 +0.18%) 등 부품사도 소폭 상승했다. 로봇주는 트럼프 대통령 취임 이전부터 '트럼프 수혜주'로 분류되며 관심을 모았다. 글로벌 공급망 재편으로 미국 내 제조 기업이 자동화 기기 도입을 가속화하면 로봇 수요가 늘어날 것이라는 전망이 우세했다. 새해 초부터는 엔비디아와 테슬라 등 글로벌 빅테크가 로봇 개발과 AI 로봇 상용화에 나서고 있다는 점도 로봇주에 대한 기대감을 높였다. 국내에서는 앞서 삼성전자가 레인보우로보틱스를 자회사로 편입했다고 밝히며 로봇 투자 확대에 나섰다. 양승윤 유진투자증권 연구원은 최근 리포트에서 ""최근 휴머노이드 양산 시기가 앞당겨지고, 국내외 주요 기업들의 로봇 사업 확대 기대감 높아지고 있는 영향으로 로봇주 강세가 지속되고 있다""며 ""다만 현재 상용화되어 판매되는 로봇 수요가 부진한 상황에는 유의가 필요하다""고 전했다."
        ]
df = pd.DataFrame(text, columns=['content'])
result_df = extract_aspect_sentences(df, std_threshold=0.4)
result_df


,raw_sentence,processed_sentence
0,한편 지난달 7일부터 LA 지역에서 발생한 2건의 산불로 최소 28명이 숨졌고 1만...,한편 지난달 7일 LA 지역 발생한 2건 산불 최소 28명숨졌고 1만 6000개 이...
1,"인공지능 열풍에 글로벌 판도가 바뀌고 있으며, 트럼프 행정부 출범에 전 세계 시장이...","인공지능 열풍 글로벌 판도 바뀌고 있으며, 트럼프 행정부 출범 전 세계 시장 출렁이..."
2,2심에서 19개 혐의에 대해 모두 무죄를 선고받은 삼성전자 이재용 회장을 대법원에 ...,2심에서 19개 혐의 대해 모두 무죄 선고받은 삼성전자 이재용 회장 대법원 상고할지...
3,"상대 진영의 척결을 알리는 신호탄으로, 권력 교체기의 길들이기 일환으로 기업인은 이...","상대 진영 척결 알리는 신호탄, 권력 교체기 길들이기 일환 기업인 이용당하기 일쑤였다."
4,법은 기업인에 대한 책임을 물으면서도 개인에 대한 형사 기소보다는 규제 조치와 금전...,법 기업인 대한 책임 물으면서도 개인 대한 형사 기소 규제 조치 금전적 처벌 주로 ...
...,...,...
60,대기업발 투자 소식에 더해 미국 내 제조시설 투자 확대 등 사업 기대감이 투자자들의...,대기업발 투자 소식 더해 미국 내 제조시설 투자 확대 등 사업 기대감 투자자들의 관...
61,7일 오전 코스닥 시장에서 레인보우로보틱스는 전일 대비 1만1000원(2.94%) ...,7일 오전 코스닥 시장 레인보우로보틱스는 전일 대비 1만1000원2.94%) 오른 ...
62,글로벌 공급망 재편으로 미국 내 제조 기업이 자동화 기기 도입을 가속화하면 로봇 수...,글로벌 공급망 재편 미국 내 제조 기업 자동화 기기 도입 가속화하면 로봇 수요 늘어...
63,새해 초부터는 엔비디아와 테슬라 등 글로벌 빅테크가 로봇 개발과 AI 로봇 상용화에...,새해 초 엔비디아 테슬라 등 글로벌 빅테크 로봇 개발 AI 로봇 상용화에 나서고 있...
